<a href="https://colab.research.google.com/github/yhk775206/AIContents/blob/main/d0_csv%ED%8C%8C%EC%9D%BC_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* WikiPainting.csv -> {.csv, .csv, .csv} 분할
### Input
* total.csv
* div_set
    * 분할 데이터 정보
    * 예) {'train': 0.70, 'val': 0.15, 'test':0.15}  
* key field
    * 클래스로 삼을 필드
    * 예) 'technique' or 'style' or 'artist_slug' or ...
* del_n
    * 이하 개수 갖는 클래스는 삭제
* limit_n
    * 각 클래스가 가질 수 있는 최대 데이터 개수
* similar classes
    * 비슷한 클래스 집합
    * 예) 'lithograph', 'lithography'
* del_classes
    * 제거할 클래스 집합
    * 예) 'woodcut'
* rand_shuffle_idx
    * 클래스 분할 전 셔플된 인덱스
    * rand(0, 전체 데이터 개수)
    * 처음에 고정시키는 게 좋음
    
### Output
* .csv 파일들
    * 예) Train.csv, Val.csv, Test.csv
* 출력값(데이터 정보)
    * (prev) 변경 전 정보
        * total cnt: 전체 데이터 개수
        * classes: 클래스 종류
        * class_cnt: 각 클래스의 데이터 개수
    * (post) 변경 후 정보
        * total cnt, classes, class_cnt

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

cwd = os.getcwd()
savepath = cwd + '/../../wiki-data_provider/' + 'images/'

In [ ]:
# path 의 csv 파일 안의 데이터 개수만 리턴
def GetN(i_total_csv_path):
    with open(i_total_csv_path) as csvfile:
        info = pd.read_csv(csvfile)
        return len(info)
    return 0

# * i_info: .csv 로부터 읽은 자료
def GetImgFilename(i_info, idx):
    image_id = i_info['image_id'][idx]
    imgfilename = savepath + image_id + ".jpg"  
    return imgfilename

In [ ]:
# * 클래스별 원소 개수 세기
# * 파라미터
#  ㄴi_info: .csv 파일로 읽은 정보
#  ㄴi_key_field: 클래스 분류 기준 (예: 'technique')
#  ㄴi_invalid_arr: 무효파일 인덱스 리스트
def CountClassCnt(i_info, i_key_field, i_invalid_arr):
    class_cnt = dict()
    for i in range(len(i_info)):
        if i in i_invalid_arr:
            continue
        
        key_val = i_info[i_key_field][i]
        
        if key_val not in class_cnt:
            class_cnt[key_val] = 1
        else:
            class_cnt[key_val] += 1
    return class_cnt

# 모든 클래스의 모든 원소의 개수의 합
def SumClassCnt(i_class_cnt):
    mysum = 0
    for val in i_class_cnt:
        mysum += i_class_cnt[val]
    return mysum

In [ ]:
# info의 field 값을 바꾼다: from_val -> to_val
# 예) info[i]['technique']: 'oil on canvas' ->'oil'
def ChangeFieldVal(i_info, i_key_field, 
                   from_val, to_val):
    for i in range(len(i_info)):  # 모든 데이터에 대해
        curr_val = i_info[i_key_field][i]
        if (curr_val==from_val):
            i_info[i_key_field][i] = to_val
            
# 중복 클래스 병합
# * 병합할 클래스 원소개수(to_val) += 병합될 클래스 원소 개수(from_val)
# 병합될 클래스 원소 개수 0 
def MergeSimilarClass(i_info, i_key_field, 
                      i_similar_classes):     
    # 각 리스트의 [0] 로 다른 클래스의 개수 합치기
    for myset in i_similar_classes:
        to_val = myset[0]        # 커질 클래스
        for i in range(1, len(myset)):  # index 1 ~ 끝
            from_val = myset[i]  # 없어질 클래스
            ChangeFieldVal(i_info, i_key_field, from_val, to_val)  # 모든 클래스에 대해 필드값 변경
            
            
# n개 이하 클래스 제거된 '목록' 제작
def MakeClassDelUnderN(i_class_cnt, i_del_n, i_limit_n, i_del_classes):
    cnt = 0
    classes = []
    dummy = dict(i_class_cnt)  # dummy
    for i, c in enumerate(dummy):
        if ( (i_class_cnt[c]<i_del_n)  # 100 개 이하
            or (type(c)!=str)          # nan(type 부재)
            or (c in i_del_classes)):    # 제거된 클래스 목록에 있음
            
            del i_class_cnt[c]  # 제거
            continue
        if (i_class_cnt[c]>=i_limit_n):  # 클래스의 데이터 개수 제한   여기!!
            print("%s class: %d -> %d" %(c, i_class_cnt[c], i_limit_n))
            i_class_cnt[c] = i_limit_n
        classes.append(c)
        cnt += 1
    nclass = cnt
        
    classes.sort()         # class 이름순 정렬
        
    class_to_idx = dict()  # class_to_idx 설정
    for i, c in enumerate(classes):
        class_to_idx[c] = i
    
    return classes, nclass, class_to_idx

In [ ]:
# * Divide data into div_set
# * 파라미터
#  ㄴi_invalid_arr: 무효파일 인덱스 리스트
# 주의: % 연산을 각 클래스에 대해서 하므로, 숫자가 깔끔히 떨어지지 않는 경우가 있음. 이 경우, train 데이터로 할당함.
def DivideData(i_info, i_div_set, i_key_field,
               i_limit_n,
               i_class_cnt,
               i_rand_shuff=None,
               i_invalid_arr=None):
    print("Divide Data(total:%d).." %len(i_info))
    
    nset = len(i_div_set)
    # 1. set_cnt, set_info 구성    
    set_cnt = dict()
    set_info = dict()
    for dset in i_div_set:      # {'train', 'val', 'test'}
        set_cnt[dset] = dict()
        set_info[dset] = list()
    
    # 2. 모든 info 마다 set 지정. set_cnt++, set_info update
    if ( type(i_rand_shuff)==np.ndarray ):
        fordummy = i_rand_shuff
    else:
        fordummy = range(len(i_info))
    
    #for i in range(len(i_info)):
    for i in fordummy:
        if ( type(i_invalid_arr)==list ):
            if i in i_invalid_arr:    # 무효파일 체크
                continue
            
        row = dict()  # 한 개 데이터 정보
        for c in i_info.columns:  # 'image_id', 'artist_slug', ...
            row[c] = i_info[c][i]  # c: field, i: index
            
        # 거르기 1: 유효파일 검사
#         imgfilename = GetImgFilename(i_info, i)
#         is_val_file = IsValidFile(imgfilename)    
#         if (is_val_file==False):
#             continue        
        # 거르기 2: nan
        key_val = i_info[i_key_field][i]  # 예) technique 中 1
        if (type(key_val)!=str):  
            continue
        # 거르기 3: custumized class 에 없다면
        if key_val not in i_class_cnt:
            continue
        
        # {Train, Valid, Test} 셋으로 나누기
        allocated = False
        for dset in i_div_set:      
            if key_val not in set_cnt[dset]:
                set_cnt[dset][key_val] = 0
            
            target_n = int( float(i_class_cnt[key_val])*i_div_set[dset] + 0.5 )  # 목표 수집 데이터 개수(예: train=total*0.7)
            if ( set_cnt[dset][key_val] >= target_n ):
                continue
            set_cnt[dset][key_val] += 1
            set_info[dset].append(row)
            
            allocated = True
            #print("\n[%d] %s" %(i, dset))
            #print("%s: %d/%d (total: %d)" %(key_val, set_cnt[dset][key_val], target_n, i_class_cnt[key_val]))
            break  # 어딘가 할당 됐으면 바로 나가기
                    
#         if (allocated==False):  # % 계산으로 인해 {test, train, val}아무 곳도 할당되지 않았다면 -> train
#             set_cnt['train'][key_val] += 1
#             set_info['train'].append(row)
#             #print("\n!Not allocated [%d] %s" %(i, dset))
#             #print("%s: %d/%d (total: %d)" %(key_val, set_cnt[dset][key_val], target_n, i_class_cnt[key_val]))
    
    # 각 {test, train, val} 의 각 클래스 별 원소 개수 프린팅
    #all_classes = set(i_class_cnt)
    mysum = 0
    for dset in i_div_set:
        nset = len(set_info[dset])
        mysum += nset
        print("%s: %d" %(dset, nset))
#         # 모든 클래스에 대해
#         for key_val in all_classes:
#             print("ㄴ%s: %d -> %d" %(key_val, i_class_cnt[key_val], set_cnt[dset][key_val]))
    print("Total: %d" %mysum)
        
    # 3. return set_info
    return set_info

In [ ]:
from PIL import Image
import imageio

# file name -> 유효파일 검사
def IsValidFile(fname):
    # 오픈될 수 있는 파일인지
    try:
        Image.open(fname)
    except:
        return False
    
#     # shape 이 (wid, hei, 3) 인지
#     try:
#         currimg = imageio.imread(fname)
#     except:
#         return False
    
#     #currimg = imread(fname)
#     if ( len(currimg.shape)!=3 or currimg.shape[2]!=3 ):
#         return False
    
    return True

In [ ]:
# 무효 파일 목록 제작
def MakeInvalList(i_info):
    n = len(i_info) # 여기
    print("Invalid Checking..(Total %d)" %n)
    invalid_arr = []
    
    inv_fname = "invalid_idx.txt"
    
    # option 1. 파일읽기
    if os.path.isfile(inv_fname):
        print("Read file: %s" %inv_fname)
        f = open(inv_fname)
        lines = f.readlines()
        for line in lines:
            tmp = int(line)
            invalid_arr.append(tmp)
            print("%d," %tmp),
        
    # option 2. 계산 + 파일 쓰기
    else:
        f = open(inv_fname, 'w')
        for i in range( n ):
            fname = GetImgFilename(i_info, i)
            isval = IsValidFile(fname)
            if(isval==False):
                data = "%d\n" %i
                f.write(data)
                invalid_arr.append(i)
                print("%d," %i),
                #print("invaild: %d %s" %(i, fname))
    f.close()
    
    print("\n무효파일: %d 개" %len(invalid_arr))
    return invalid_arr

# 무효파일 아닌 파일 출력하기 (확인용)
def VisValid(i_info, i_inval_arr):
    n = len(i_info)   # 여기
    
    print("total: %d" %n)
    for i in range( n ):
        if i%100==0:
            print ("%d.." %(i)),
        if i in i_inval_arr:
            continue
            
        fname = GetImgFilename(i_info, i)
        if os.path.isfile(fname):
            img = imageio.imread(fname)
            if ( len(img.shape)!=3 or img.shape[2]!=3 ):
                plt.imshow(img)
                plt.title("[%d] %s" %(i, img.shape))            
                #plt.title("%s, %s" %(image_id, key_val))
                plt.show()
                
                print("isvalid?"),
                print(IsValidFile(fname))

In [ ]:
csv_path = cwd + '/../../wiki-data_provider/' + 'wikipaintings_oct2013.csv'
with open(csv_path) as csvfile:
    info = pd.read_csv(csvfile)
    inval_arr = MakeInvalList(info)
    print(inval_arr)

Invalid Checking..(Total 101086)
Read file: invalid_idx.txt
685, 749, 863, 1232, 1728, 1842, 2598, 2693, 3222, 3261, 5503, 5512, 5996, 6059, 6201, 6252, 6259, 6366, 6378, 6657, 6675, 6689, 7020, 7180, 7451, 7828, 7925, 8389, 8485, 8861, 9049, 9096, 9240, 9513, 9631, 9738, 10015, 10026, 10377, 10604, 10622, 11034, 11105, 11805, 11857, 11994, 12634, 12714, 12827, 12937, 12954, 13061, 13348, 13720, 13776, 13993, 14099, 14385, 15140, 16272, 16279, 16300, 16416, 16468, 16510, 16529, 16788, 17474, 17504, 17693, 18736, 18784, 19386, 19524, 19765, 19996, 20017, 20032, 20050, 20915, 20961, 20963, 21326, 21492, 22031, 22056, 22177, 22212, 22248, 22289, 22500, 22574, 22777, 23072, 23090, 23116, 23126, 23153, 23207, 23398, 23638, 23960, 24412, 24715, 24799, 24820, 24853, 24866, 25006, 25827, 25897, 26067, 26408, 26672, 27476, 27955, 28073, 28456, 28463, 28575, 28976, 29017, 29245, 29321, 29822, 30751, 31308, 32051, 32072, 32096, 32275, 32295, 32948, 33021, 33032, 33071, 33091, 33276, 33682, 33683,

In [ ]:
# VisValid(info, inval_arr)

In [ ]:
def CustumizeNDivideData(i_total_csv_path, i_div_set, i_key_field, 
               i_del_n, i_limit_n, i_similar_classes=None, i_del_classes=None, i_rand_shuff=None):
    with open(i_total_csv_path) as csvfile:
        info = pd.read_csv(csvfile)
        
        # 0. 무효파일 목록 제작
        print("=== 0. 무효파일 목록 제작 or 읽기 ===")
        inval_arr = MakeInvalList(info)
        
        # 1. 모든 클래스 종류 & 각 클래스 원소 개수
        print("=== 1. 모든 클래스 종류 & 각 클래스 원소 개수 ===")
        prev_classes = set(info[i_key_field])  # 모든 클래스 종류
        prev_nclass = len(prev_classes)
        prev_class_cnt = CountClassCnt(info, i_key_field, inval_arr)
        print("=== Prev data ===")
        print("# of all data: %d" %SumClassCnt(prev_class_cnt))
        print("# of classes: %d" %prev_nclass)
        print("--- # of each class elements ---")
        print(prev_class_cnt)
        
        # 2. 중복 클래스 병합
        print("\n=== 2. 중복 클래스 병합 & 각 클래스 원소 개수 ===")
        post_class_cnt = dict(prev_class_cnt)
        MergeSimilarClass(info, i_key_field, i_similar_classes)
        # +) 중복 클래스 병합 후 각 클래스 원소 개수 파악
        post_class_cnt = CountClassCnt(info, i_key_field, inval_arr)
        print(post_class_cnt)        
        
        # 3. {'n개 이하 클래스', 'nan 클래스', '지정된 클래스'} 제거된 '목록' 제작
        print("\n=== 3. {'n개 이하 클래스', 'nan 클래스', '지정된 클래스'} 제거된 '목록' 제작 ===")
        post_classes, post_nclass, class_to_idx = MakeClassDelUnderN(post_class_cnt, i_del_n, i_limit_n, i_del_classes)
        print("# of all data: %d" %SumClassCnt(post_class_cnt))
        print("# of classes: %d" %post_nclass)
        print("--- # of each class elements ---")
        print(post_class_cnt)
        print("--- class_to_idx ---")
        print(class_to_idx)
        
        # 4. div_set 으로 나누기
        print("\n=== 4. div_set 으로 나누기 ===")
        set_info = DivideData(info, div_set, key_field, i_limit_n, post_class_cnt, i_rand_shuff, inval_arr)
                
        return info, set_info

### 1. Parameter setting

In [ ]:
# csv_path = cwd + '/../wiki-data_provider/' + 'wikipaintings_oct2013.csv'
# div_set = {'train': 0.70, 'val': 0.15, 'test':0.15}
# #div_set = {'train': 1.0, 'val': 0.0, 'test':0.0}
# key_field = 'technique'
# del_n = 600  #80
# limit_n = 1000
# similar_classes = [['oil', 'oil on copper', 'oil on canvas', 'oil on panel'],
#                   ['pencil', 'colored pencils'],
#                   ['watercolor', 'watercolour'],
#                   ['lithograph', 'lithography'],
#                   ['tempera', 'tempera on canvas', 'egg tempera on panel'],
#                   ['ink', 'indian ink']]
# del_classes = ['woodcut', 'woodblock print', 
#                'etching', 'lithograph']   # 염료를 통해 스트로크를 남기는 media 가 아님. 부식 동판술, 석판화.

# is_shuffle = True
# rand_shuff_idx = None
# if is_shuffle:
#     n = GetN(csv_path)
#     rand_shuff_idx = np.random.choice(n, n, replace=False)   # 0~n 사이 중 n 개 골라라(중복불허)

In [ ]:
csv_path = cwd + '/../../wiki-data_provider/' + 'wikipaintings_oct2013.csv'
div_set = {'train': 0.70, 'val': 0.15, 'test':0.15}
#div_set = {'train': 1.0, 'val': 0.0, 'test':0.0}
key_field = 'technique'
del_n = 230
limit_n = 1000
similar_classes = [['oil', 'oil on copper', 'oil on canvas', 'oil on panel'],
                  ['pencil', 'colored pencils'],
                  ['watercolor', 'watercolour'],
                  ['lithograph', 'lithography'],
                  ['tempera', 'tempera on canvas', 'egg tempera on panel'],
                  ['ink', 'indian ink']]
del_classes = ['woodcut', 'woodblock print', 
               'gouache', 'woodcut', 'collage', 'chalk'
              ]

is_shuffle = True
rand_shuff_idx = None
if is_shuffle:
    n = GetN(csv_path)
    rand_shuff_idx = np.random.choice(n, n, replace=False)   # 0~n 사이 중 n 개 골라라(중복불허)

### 2. 데이터 나누기

In [ ]:
orig_info, set_info = CustumizeNDivideData(csv_path, div_set, key_field,
                                           del_n, limit_n, similar_classes, del_classes,
                                           rand_shuff_idx)

=== 0. 무효파일 목록 제작 or 읽기 ===
Invalid Checking..(Total 101086)
Read file: invalid_idx.txt
685, 749, 863, 1232, 1728, 1842, 2598, 2693, 3222, 3261, 5503, 5512, 5996, 6059, 6201, 6252, 6259, 6366, 6378, 6657, 6675, 6689, 7020, 7180, 7451, 7828, 7925, 8389, 8485, 8861, 9049, 9096, 9240, 9513, 9631, 9738, 10015, 10026, 10377, 10604, 10622, 11034, 11105, 11805, 11857, 11994, 12634, 12714, 12827, 12937, 12954, 13061, 13348, 13720, 13776, 13993, 14099, 14385, 15140, 16272, 16279, 16300, 16416, 16468, 16510, 16529, 16788, 17474, 17504, 17693, 18736, 18784, 19386, 19524, 19765, 19996, 20017, 20032, 20050, 20915, 20961, 20963, 21326, 21492, 22031, 22056, 22177, 22212, 22248, 22289, 22500, 22574, 22777, 23072, 23090, 23116, 23126, 23153, 23207, 23398, 23638, 23960, 24412, 24715, 24799, 24820, 24853, 24866, 25006, 25827, 25897, 26067, 26408, 26672, 27476, 27955, 28073, 28456, 28463, 28575, 28976, 29017, 29245, 29321, 29822, 30751, 31308, 32051, 32072, 32096, 32275, 32295, 32948, 33021, 33032, 33071,

## 3. csv 파일 쓰기

In [ ]:
import csv
# 각 set_info[test, train, val] 에 대해 csv 파일 생성
def WriteCSV(i_info, i_a_set_info, i_path):
    with open(i_path, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=i_info.columns)
        writer.writeheader()
        for row in i_a_set_info:
            writer.writerow(row)

In [ ]:
for ds in div_set:
    path = "wikipaintings_oct2013_%s+%s+%d.csv" %(key_field, ds, div_set[ds]*100)
    print(path)
    WriteCSV(orig_info, set_info[ds], path)

wikipaintings_oct2013_technique+test+15.csv
wikipaintings_oct2013_technique+train+70.csv
wikipaintings_oct2013_technique+val+15.csv


## 4. visualize

In [ ]:
from scipy.misc import imread, imresize
#savepath = cwd + '/../wiki-data_provider/' + 'images/'
def VizSample(i_info, i_key_field, i_n=1):
    #classes = set(i_info[i_key_field])
    #print(classes)
    
    visit = dict()  # 시각화 몇 개 했는 지
    for i in range(len(i_info)):
        key_val = i_info[i_key_field][i]
        if key_val not in visit:
            visit[key_val] = 1
        else:
            visit[key_val] += 1
           
        # 시각화
        if visit[key_val] <= i_n:
            imgfilename = GetImgFilename(i_info, i)
            print("%s (%d/%d)" %(key_val, visit[key_val], i_n))
            if os.path.isfile(imgfilename):
                img = imread(imgfilename)
                plt.imshow(img)
                #plt.title("%s, %s" %(image_id, key_val))
                plt.show()
# 비효율적([i] 위치만 바꿈)
def VizSample2(i_info, i_key_field, i_n=1):
    #classes = set(i_info[i_key_field])
    #print(classes)
    
    visit = dict()  # 시각화 몇 개 했는 지
    for i in range(len(i_info)):
        key_val = i_info[i][i_key_field]
        if key_val not in visit:
            visit[key_val] = 1
        else:
            visit[key_val] += 1
           
        # 시각화
        if visit[key_val] <= i_n:
            imgfilename = GetImgFilename(i_info, i)
            print("%s (%d/%d)" %(key_val, visit[key_val], i_n))
            if os.path.isfile(imgfilename):
                img = imread(imgfilename)
                plt.imshow(img)
                #plt.title("%s, %s" %(image_id, key_val))
                plt.show()

In [ ]:
with open(csv_path) as csvfile:
    info = pd.read_csv(csvfile)
    VizSample(info, key_field, 2)

In [ ]:
#set_info['train'][0]
VizSample2(set_info['train'], key_field, 2)